In [1]:
from __future__ import print_function
import os

import pandas as pd
import numpy as np
import matplotlib

# Ingestión de datos

In [2]:
data_path = ['data']
file_name = 'Vuelos.csv'
filepath = os.sep.join(data_path + [file_name])

dataset = pd.read_csv(filepath, low_memory=False)

Se presentan las primeras 5 observaciones y las dimensiones de la tabla de observaciones. Notamos que la cantidad de observaciones es alta comparado con la cantidad de features, lo cúal es muy positivo cuando se aplican modelos estadísticos.

In [3]:
display(dataset.shape)
display(dataset.head(5))

(480085, 38)

,Unnamed: 0,MONTH,DAY_OF_MONTH,FL_DATE,UNIQUE_CARRIER,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_WAC,...,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 36
0,0,8,31,2017-08-31,F9,LGA,"New York, NY",NY,New York,22,...,175.0,1.0,762.0,4,NaN,NaN,NaN,NaN,NaN,NaN
1,1,8,2,2017-08-02,WN,STL,"St. Louis, MO",MO,Missouri,64,...,NaN,1.0,888.0,4,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8,2,2017-08-02,WN,STL,"St. Louis, MO",MO,Missouri,64,...,138.0,1.0,888.0,4,NaN,NaN,NaN,NaN,NaN,NaN
3,3,8,2,2017-08-02,WN,STL,"St. Louis, MO",MO,Missouri,64,...,166.0,1.0,888.0,4,0.0,39.0,26.0,0.0,44.0,NaN
4,4,8,2,2017-08-02,WN,TPA,"Tampa, FL",FL,Florida,33,...,NaN,1.0,1130.0,5,NaN,NaN,NaN,NaN,NaN,NaN


En lo anterior se observa la presencia de dos features sin nombre, "Unnamed: 0" y "Unnamed: 36". El caso de la primera variable parece cumplir una función de orden, por lo que es inecesaria pues ya contamos con los indices del dataframe, por lo que si confirmamos esto entonces esta columna podría eliminarse. El caso de la segunda variable lo analizaremos más adelante cuando contemos con más información.

In [4]:
print(pd.DataFrame(dataset['Unnamed: 0']).equals(np.linspace(0,480084,480084,dtype=int))) #Deberia dar True
print('\n')
dataset.info()

False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480085 entries, 0 to 480084
Data columns (total 38 columns):
Unnamed: 0             480085 non-null int64
MONTH                  480085 non-null int64
DAY_OF_MONTH           480085 non-null int64
FL_DATE                480085 non-null object
UNIQUE_CARRIER         480085 non-null object
ORIGIN                 480085 non-null object
ORIGIN_CITY_NAME       480085 non-null object
ORIGIN_STATE_ABR       480085 non-null object
ORIGIN_STATE_NM        480085 non-null object
ORIGIN_WAC             480085 non-null int64
DEST                   480085 non-null object
DEST_CITY_NAME         480085 non-null object
DEST_STATE_ABR         480085 non-null object
DEST_STATE_NM          480085 non-null object
DEST_WAC               480085 non-null int64
DEP_TIME               468246 non-null float64
DEP_DELAY              468236 non-null float64
DEP_DELAY_NEW          468236 non-null float64
DEP_DEL15              468236 non-null float64
DEP_DELA

Se observa que "Unnamed: 0" de hecho si cumplía una función de enumeración por lo se procede a eliminar, además "Unnamed: 36" posee 0 valores no nulos, por lo que se concluye que es imposible conseguir información reelevante de esta columna y se eliminará.
Un patrón importante es que las variables relacionadas con motivo de atraso son aquellas que presentan mayor cantidad de valores nulos, más aún, todas tienen exactamente la misma cantidad de valores nulos, esto nos lleva a considerar la posibilidad que un valor nulo signifique enrealidad que no hubo ningún atraso por lo que no se registra la causa, esto lo confirmamos a continuación.

In [5]:
#Eliminamos las columnas que no aportan información
dataset.drop(['Unnamed: 36','Unnamed: 0'], axis=1, inplace=True)

#Agregar función que confirme en su gran mayoría cuando delay es positivo entonces los valores de los motivos son no-nulos
mascara = dataset['ARR_DELAY']>0 
mascara2 = dataset['DEP_DELAY']>0
mascara3 = mascara | mascara2
dataset[mascara].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169840 entries, 0 to 480084
Data columns (total 36 columns):
MONTH                  169840 non-null int64
DAY_OF_MONTH           169840 non-null int64
FL_DATE                169840 non-null object
UNIQUE_CARRIER         169840 non-null object
ORIGIN                 169840 non-null object
ORIGIN_CITY_NAME       169840 non-null object
ORIGIN_STATE_ABR       169840 non-null object
ORIGIN_STATE_NM        169840 non-null object
ORIGIN_WAC             169840 non-null int64
DEST                   169840 non-null object
DEST_CITY_NAME         169840 non-null object
DEST_STATE_ABR         169840 non-null object
DEST_STATE_NM          169840 non-null object
DEST_WAC               169840 non-null int64
DEP_TIME               169840 non-null float64
DEP_DELAY              169840 non-null float64
DEP_DELAY_NEW          169840 non-null float64
DEP_DEL15              169840 non-null float64
DEP_DELAY_GROUP        169840 non-null float64
DEP_TIME_BLK  

Observe que cuando consideramos el subconjunto de los datos en los que hubo un atraso, la diferencia entre el total de los datos y la cantidad de valores no nulos de las variables con motivos de atraso disminuye considerablemente, aquellos datos que siguen teniendo valores nulos probablemente es porque no se registró la causa del retraso.

In [6]:
#Observamos el resumen general de las observaciones que tenemos
#Agregar grafico de barras o histograma
dataset.describe()

,MONTH,DAY_OF_MONTH,ORIGIN_WAC,DEST_WAC,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,ARR_TIME,...,DIVERTED,ACTUAL_ELAPSED_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
count,480085.000000,480085.000000,480085.000000,480085.000000,468246.000000,468236.000000,468236.000000,468236.000000,468236.000000,467602.000000,...,480085.000000,466352.000000,480085.0,480085.000000,480085.000000,107030.000000,107030.000000,107030.000000,107030.000000,107030.000000
mean,6.522522,15.775219,34.340188,34.339329,1338.094262,15.403781,18.434883,0.228346,0.354919,1462.393080,...,0.003114,175.038833,1.0,1068.561001,4.750365,21.397132,2.799981,25.117416,0.113856,26.045772
std,3.404854,8.766987,21.222605,21.220896,516.176915,54.283760,53.125508,0.419767,2.649006,577.113661,...,0.055717,89.470467,0.0,717.577157,2.759537,65.068146,23.860263,48.045493,2.968468,51.109784
min,1.000000,1.000000,2.000000,2.000000,1.000000,-84.000000,0.000000,0.000000,-2.000000,1.000000,...,0.000000,37.000000,1.0,75.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,8.000000,22.000000,22.000000,908.000000,-5.000000,0.000000,0.000000,-1.000000,1038.000000,...,0.000000,112.000000,1.0,544.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,16.000000,22.000000,22.000000,1336.000000,-2.000000,0.000000,0.000000,-1.000000,1512.000000,...,0.000000,155.000000,1.0,950.000000,4.000000,0.000000,0.000000,8.000000,0.000000,0.000000
75%,9.000000,23.000000,36.000000,36.000000,1749.000000,11.000000,11.000000,0.000000,0.000000,1934.000000,...,0.000000,210.000000,1.0,1389.000000,6.000000,18.000000,0.000000,29.000000,0.000000,31.000000
max,12.000000,31.000000,93.000000,93.000000,2400.000000,1685.000000,1685.000000,1.000000,12.000000,2400.000000,...,1.000000,784.000000,1.0,4983.000000,11.000000,1548.000000,1112.000000,1276.000000,292.000000,1240.000000


In [7]:
#Contamos la cantidad de vuelos segun ciudad de salida
#Agregar grafico de barras o histograma

dataset.groupby("ORIGIN_CITY_NAME").size().to_frame().head(5)

,0
ORIGIN_CITY_NAME,
"Aguadilla, PR",459
"Albany, NY",9403
"Albuquerque, NM",313
"Asheville, NC",10
"Atlanta, GA",17720


In [8]:
#Contamos la cantidad de vuelos segun ciudad de destino
#Agregar grafico de barras o histograma
dataset.groupby("DEST_CITY_NAME").size().to_frame().head(5)

,0
DEST_CITY_NAME,
"Aguadilla, PR",462
"Albany, NY",9401
"Albuquerque, NM",313
"Asheville, NC",10
"Atlanta, GA",17717


In [9]:

#Contamos al cantidad de vuelos cancelados

dataset.groupby("CANCELLED").size().to_frame()


,0
CANCELLED,
0.0,467842
1.0,12243
